In [ ]:
#@title **User Creation**
#@markdown Enter your preferred username and password for the server.

import os, sys

username = "user" #@param {type:"string"}
password = "colab" #@param {type:"string"}
shell = "bash" #@param ["bash", "sh", "zsh"]
#@markdown If you selected `zsh`   as your shell, optionally add your dotfiles Git repository here.
dotfiles = "sdkfjd" #@param {type:"string"}

if dotfiles != "":
  try:
    if dotfiles.split(".")[1].split(".")[1] != ".git":
        print("Please make sure this is a clonable repository (.git at the end of the line).")
        dotfilesExist = False
    else:
        dotfilesExist = True
  except:
      print("Please make sure this is a clonable repository (.git at the end of the line).")
      dotfilesExist = False
else:
  print("Creating and configuring...")

  if dotfilesExist is True:
    # Install git
    os.system("apt install --assume-yes git")

  # Create user
  os.system(f"useradd -m {username}")

  # Add user to 'sudo' group (superuser access)
  os.system(f"adduser {username} sudo")
      
  # Set password of user to chosen variable
  os.system(f"echo '{username}:{password}' | sudo chpasswd")

  # Installing shell
  os.system(f"apt install --assume-yes {shell}")

  # Change default shell from sh to chosen shell
  os.system(f"sed -i 's/\/bin\/sh/\/bin\/{shell}/g' /etc/passwd")

  print("User creation and configuration succeeded!")

Please make sure this is a clonable repository (.git at the end of the line).
Creating and configuring...
User creation and configuration succeeded!


In [ ]:
#@title **RDP**
#@markdown  This will normally take 3-5 minutes. Please be patient.

import os
import subprocess

#@markdown  Visit https://remotedesktop.google.com/headless and copy the entire command after you authenticate. Please paste the command here.

CRP = "" #@param {type:"string"}

#@markdown Please enter a pin more or equal to 6 digits.
Pin = 102938 #@param {type: "integer"}

#@markdown Please select what desktop environment you want to use.
Desktop = 'XFCE' #@param ["XFCE", "KDE Plasma", "Gnome"]

if str(Desktop) == "XFCE":
    desktopInstall = "xfce4 desktop-base xfce4-terminal"
    xserverStart = "/usr/bin/xfce4-session"
    displayManager = "lightdm"
elif str(Desktop) == "KDE Plasma":
    desktopInstall = "kde-plasma-desktop kde-runtime konsole"
    xserverStart = "/usr/bin/plasmashell"
    displayManager = "sddm"
elif str(Desktop) == "Gnome":
    desktopInstall = "ubuntu-gnome-desktop gnome-shell"
    xserverStart = "/usr/bin/gnome-session"
    displayManager = "lightdm"

class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChrome()
        self.finish()

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop...")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing your chosen desktop environment...")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system(f"apt install --assume-yes {desktopInstall}")
        os.system(f"bash -c 'echo \"exec /etc/X11/Xsession {xserverStart}\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system(f"systemctl disable {displayManager}.service")

    @staticmethod
    def installGoogleChrome():
        print("Installing Google Chrome...")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish():
        print("Finalizing...")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Successfully finished.")

try:
    if username:
        if CRP == "":
            print("Please enter your authentication script from the given link into the 'CRP' field.")
        elif len(str(Pin)) < 6:
            print("Please enter a pin more or equal to 6 digits in the 'Pin' field.")
        else:
            CRD()
except NameError as e:
    print("Username not set.")
    print("Please create a user before running.")

In [ ]:
#@title **SSH**

! pip install colab_ssh --upgrade &> /dev/null

Ngrok = False #@param {type:'boolean'}
Agro = True #@param {type:'boolean'}


#@markdown Copy the authentication token from https://dashboard.ngrok.com/auth (only for Ngrok, not for Agro)
NgrokToken = "" #@param {type:'string'}


def runNGROK():
    from colab_ssh import launch_ssh
    from IPython.display import clear_output
    launch_ssh(ngrokToken, password)
    clear_output()

    print("ssh", username, end='@')
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
            "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"


def runAgro():
    from colab_ssh import launch_ssh_cloudflared
    launch_ssh_cloudflared(password=password)


try:
    if username:
        pass
    elif password:
        pass
except NameError:
    print("No user has been set. Using the default username 'user' and password 'colab'.")
    username='user'
    password='colab'


if Agro and Ngrok:
    print("You can't use both.")
    print("Please select only one of them.")
elif Agro:
    runAgro()
elif Ngrok:
    if NgrokToken == "":
        print("No Ngrok token found, please enter it")
    else:
        runNGROK()
else:
    print("Please select one of the options.")

In [ ]:
#@title **Colab Shutdown**

#@markdown Shutdown Ngrok Tunnel
Ngrok = False #@param {type:'boolean'}

#@markdown Unmount Google Drive
GoogleDrive = False #@param {type:'boolean'}

#@markdown Stop Colab
Sleep = True #@param {type:'boolean'}

if Ngrok:
    ! killall ngrok

if GoogleDrive:
    with open('/content/unmount.py', 'w') as unmount:
        unmount.write("""from google.colab import drive
drive.flush_and_unmount()""")
    
    try:
        if user:
            ! runuser $user -c 'python3 /content/unmount.py'
    except NameError:
        print("Google Drive not Mounted")

if Sleep:
    from time import sleep
    sleep(43200)